In [1]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# 셀리움 import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
!pip install selenium

In [4]:
# bs4 / 404 해결
import urllib.request
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
import urllib.request

In [5]:
options = webdriver.ChromeOptions()

# 크롬 파일 저장 시 경로 설정
options.add_experimental_option(
	"prefs", {
		"download.default_directory": r"//Users/hwanghyeonjun/Documents/GitHub/data/selenium",
		"download.prompt_for_download": False,
		"download.directory_upgrade": True,
		"safebrowsing.enabled": True
	}
)

# 크롬 시크릿 모드로 실행
options.add_argument('incognito')

# user agent 설정
user_agent = "Chrome/71.0.3578.83"
options.add_argument('user-agent=' + user_agent)

In [6]:
# MACOS 크롬 드라이버
driver = webdriver.Chrome('./driver/chromedriver', options=options)
driver.maximize_window()

/var/folders/xz/4cx3hhnx0bsgz2sth0876md80000gn/T/ipykernel_28627/4141595889.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver', options=options)


In [7]:
'''
	start_date : 시작 날짜

  calender_btn : 과거 버전 캘린더 버튼
  calender_new_btn : 최신 버전 페이지 캘린더 버튼

  start_year_input : 과거 버전 캘린더 인풋
  start_year_new_input : 최신 버전 캘린더 인풋

  apply_btn : 과거 버전 적용 버튼
  apply_new_btn : 최신 적용 버튼

  csv_download_btn : 과거 버전 다운로드 버튼
  csv_download_new_btn : 최신 버전 다운로드 버튼

  =======================================================
  임시로 주석 / 홈페이지 모바일로 되있다면 주석 풀고 실행(셀리움 코드 실행 셀에도 주식 풀기)
  bottom_modal : 홍페이지버전이 모바일 시 하단 모달
  bottom_mpdal_btn : 모달 닫기 버튼
'''
start_date = '2020-01-01'

calender_btn = '.DatePickerWrapper_icon-wrap__cwTu_'
calender_new_btn = '#flatDatePickerCanvasHol #datePickerIconWrap'

start_year_input = '.NativeDateInput_root__wbgyP > input'
start_year_new_input = '#startDate'

apply_btn = 'HistoryDatePicker_apply-button__fPr_G'
apply_new_btn = '#applyBtn'

csv_download_btn = '.download-data_download-data__jxNYT > a'
csv_download_new_btn = '#column-content > div.float_lang_base_2.downloadDataWrap > div > a'

bottom_modal = '.download-app-view_download-app-view__4d6Eq'
bottom_mpdal_btn = '.inv-button'

In [8]:
# 주요 환율 리스트
names = ['USD', 'JPY', 'EUR', 'CNY', 'HKD', 'GBP', 'CHF', 'CAD', 'AUD', 'NZD', 'SEK', 'DKK', 'NOK', 'SAR', 'KWD', 'BHD',
         'AED', 'THB', 'SGD', 'IDR', 'INR', 'MYR', 'PKR', 'BDT', 'PHP', 'EGP', 'MXN', 'BND']

In [9]:
# 원자재 리스트
list = ['gold', 'crude-oil', 'silver', 'natural-gas', 'copper', 'us-wheat']

In [10]:
# 테스트 코드들
# url = f'https://kr.investing.com/currencies/jpy-krw-historical-data'
# driver.get(url)
# driver.implicitly_wait(30)
# driver.find_element(By.CSS_SELECTOR, calender_btn).click()
# driver.find_element(By.CSS_SELECTOR, start_year_input).clear()
# driver.implicitly_wait(1)
# driver.find_element(By.CSS_SELECTOR, start_year_input).send_keys(start_date)
# driver.implicitly_wait(5)
# driver.find_element(By.CLASS_NAME, apply_btn).click()
# time.sleep(10)
# driver.find_element(By.CSS_SELECTOR, csv_download_btn).click()
# driver.implicitly_wait(30)

In [11]:
# 과거 데이터 크롤링 함수
'''
	인베스팅 과거 버전 페이지 크롤링 함수
'''


def investing_crawling(middel_url, names, start_date, suffix=''):
	suffix_ = suffix
	print('전체 개수 {0}개'.format(len(names)))

	succeed = []
	failed = []

	# 셀리움 실행 코드
	for name in tqdm(names):
		name_ = name.lower()
		try:
			url = f'https://kr.investing.com/{middel_url}/{name_}{suffix_}-historical-data'

			driver.get(url)
			headers = {'User-Agent': 'Chrome/107.0.5304.87'}
			req = urllib.request.Request(url, headers=headers)
			urlopen(req)
			time.sleep(5)

			# 모바일 화면시 주석 풀고 실행
			# is_displayed_modal = driver.find_element(By.CSS_SELECTOR, bottom_modal).is_displayed()
			# if is_displayed_modal:
			# 	driver.find_element(By.CSS_SELECTOR, bottom_mpdal_btn).click()
			# 	driver.implicitly_wait(10)

			driver.find_element(By.CSS_SELECTOR, calender_btn).click()
			driver.find_element(By.CSS_SELECTOR, start_year_input).clear()
			driver.implicitly_wait(1)
			driver.find_element(By.CSS_SELECTOR, start_year_input).send_keys(start_date)
			driver.implicitly_wait(5)
			driver.find_element(By.CLASS_NAME, apply_btn).click()
			time.sleep(3)
			driver.find_element(By.CSS_SELECTOR, csv_download_btn).click()
			driver.implicitly_wait(30)
			succeed.append(name_)
		except HTTPError as e:
			failed.append(name_)

	print('=' * 150)
	print('성공 :{0}\nerror 개수: {1}개'.format(succeed, len(succeed)))
	print('=' * 150)
	print('실패 :{0}\nerror 개수: {1}개'.format(failed, len(failed)))
	print('=' * 150)

In [12]:
# 과거 데이터 크롤링 함수
'''
	인베스팅 최신 버전 페이지 크롤링 함수
'''


def investing_crawling_new(middel_url, names, start_date, suffix=''):
	suffix_ = suffix
	print('전체 개수 {0}개'.format(len(names)))

	succeed = []
	failed = []

	# 셀리움 실행 코드
	for name in tqdm(names):
		name_ = name.lower()
		try:
			url = f'https://kr.investing.com/{middel_url}/{name_}{suffix_}-historical-data'

			driver.get(url)
			headers = {'User-Agent': 'Chrome/107.0.5304.87'}
			req = urllib.request.Request(url, headers=headers)
			urlopen(req)
			time.sleep(5)

			# 모바일 화면시 주석 풀고 실행
			# is_displayed_modal = driver.find_element(By.CSS_SELECTOR, bottom_modal).is_displayed()
			# if is_displayed_modal:
			# 	driver.find_element(By.CSS_SELECTOR, bottom_mpdal_btn).click()
			# 	driver.implicitly_wait(10)

			driver.find_element(By.CSS_SELECTOR, calender_new_btn).click()
			driver.find_element(By.CSS_SELECTOR, start_year_new_input).clear()
			driver.implicitly_wait(1)
			driver.find_element(By.CSS_SELECTOR, start_year_new_input).send_keys(start_date)
			driver.implicitly_wait(5)
			driver.find_element(By.CSS_SELECTOR, apply_new_btn).click()
			time.sleep(3)
			driver.find_element(By.CSS_SELECTOR, csv_download_new_btn).click()
			driver.implicitly_wait(30)
			succeed.append(name_)
		except HTTPError as e:
			failed.append(name_)

	print('=' * 150)
	print('성공 :{0}\n성공 개수: {1}개'.format(succeed, len(succeed)))
	print('=' * 150)
	print('실패 :{0}\nerror 개수: {1}개'.format(failed, len(failed)))
	print('=' * 150)

In [13]:
investing_crawling('currencies', names, '2020-01-01', '-krw')

전체 개수 28개


 25%|██▌       | 7/28 [01:58<05:55, 16.92s/it]


KeyboardInterrupt: 

In [24]:
investing_crawling('commodities', list, start_date)

전체 개수 6개


  0%|          | 0/6 [00:09<?, ?it/s]


ElementNotInteractableException: Message: element not interactable: element has zero size
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C8ACD3+2075859]
	Ordinal0 [0x00C1EE61+1633889]
	Ordinal0 [0x00B1B7BD+571325]
	Ordinal0 [0x00B4C059+770137]
	Ordinal0 [0x00B41C76+728182]
	Ordinal0 [0x00B6731C+881436]
	Ordinal0 [0x00B415BF+726463]
	Ordinal0 [0x00B67534+881972]
	Ordinal0 [0x00B7B56A+963946]
	Ordinal0 [0x00B67136+880950]
	Ordinal0 [0x00B3FEFD+720637]
	Ordinal0 [0x00B40F3F+724799]
	GetHandleVerifier [0x00F3EED2+2769538]
	GetHandleVerifier [0x00F30D95+2711877]
	GetHandleVerifier [0x00D1A03A+521194]
	GetHandleVerifier [0x00D18DA0+516432]
	Ordinal0 [0x00C2682C+1665068]
	Ordinal0 [0x00C2B128+1683752]
	Ordinal0 [0x00C2B215+1683989]
	Ordinal0 [0x00C36484+1729668]
	BaseThreadInitThunk [0x7570FA29+25]
	RtlGetAppContainerNamedObjectPath [0x774A7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x774A7B8E+238]


In [28]:
# 드라이버 종료
driver.close()

In [ ]:
# 세계지수 리스트 크롤링
url = 'https://kr.investing.com/indices/major-indices'

In [ ]:
try:
	headers = {'User-Agent': 'Chrome/107.0.5304.87'}
	req = urllib.request.Request(url, headers=headers)
	html = urlopen(req)
except HTTPError as e:
	err = e.read()
	code = e.getcode()
	print(err, code)

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
stock_link = soup.select('td.datatable_cell__0y0eu.datatable_cell--name__5g25Q.table-browser_col-name__qzN_9 > div > a')
stock_link

[<a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/kospi" title="코스피지수">코스피지수</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/kospi-50" title="코스피 50">코스피 50</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/kosdaq" title="코스닥">코스닥</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/us-30" title="다우존스">다우존스</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/us-spx-500" title="S&amp;P 500 - (CFD)">S&amp;P 500</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/nasdaq-composite" title="나스닥종합지수">나스닥종합지수</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/smallcap-2000" title="러셀 2000 지수 - (CFD)">러셀 2000</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/volatility-s-p-500" title="CBOE Volatility Index - (CFD)">CBOE VIX</a>,
 <a class="inv-link bold datatable_cell--

In [ ]:
indices = []
for each in stock_link:
	href = each['href']
	indice = href.split('/')[-1]
	indices.append(indice)

In [ ]:
print(indices)

['kospi', 'kospi-50', 'kosdaq', 'us-30', 'us-spx-500', 'nasdaq-composite', 'smallcap-2000', 'volatility-s-p-500', 's-p-tsx-composite', 'bovespa', 'ipc', 'germany-30', 'uk-100', 'france-40', 'eu-stoxx50', 'netherlands-25', 'spain-35', 'it-mib-40', 'switzerland-20', 'psi-20', 'bel-20', 'atx', 'omx-stockholm-30', 'mcx', 'rtsi', 'wig-20', 'hungary-stock-market', 'ise-100', 'ta25', 'tasi', 'japan-ni225', 'aus-200', 'shanghai-composite', 'szse-component', 'ftse-china-a50', 'dj-shanghai', 'hang-sen-40', 'taiwan-weighted', 'thailand-set', 'idx-composite', 's-p-cnx-nifty', 'sensex', 'vn-30']


In [ ]:
investing_crawling('indices', indices, '2020-01-01')

In [ ]:
# 암호화폐 크롤링
coins = [['bitcoin', 'btc-krw'], ['ethereum', 'eth-krw'], ['xrp', 'xrp-krw']]

In [ ]:
for coin in tqdm(coins):
	try:
		url = f'https://kr.investing.com/crypto/{coin[0]}/{coin[1]}-historical-data'

		driver.get(url)
		headers = {'User-Agent': 'Chrome/107.0.5304.87'}
		req = urllib.request.Request(url, headers=headers)
		urlopen(req)
		time.sleep(5)
		driver.find_element(By.CSS_SELECTOR, calender_btn).click()
		driver.find_element(By.CSS_SELECTOR, start_year_input).clear()
		driver.implicitly_wait(1)
		driver.find_element(By.CSS_SELECTOR, start_year_input).send_keys(start_date)
		driver.implicitly_wait(5)
		driver.find_element(By.CLASS_NAME, apply_btn).click()
		time.sleep(3)
		driver.find_element(By.CSS_SELECTOR, csv_download_btn).click()
		driver.implicitly_wait(30)
	except HTTPError as e:
		pass

100%|██████████| 3/3 [00:34<00:00, 11.61s/it]


In [ ]:
driver.close()

In [10]:
# 채권
url = 'https://kr.investing.com/rates-bonds/world-government-bonds'

In [11]:
driver.get(url)

In [12]:
try:
	headers = {'User-Agent': 'Chrome/107.0.5304.87'}
	req = urllib.request.Request(url, headers=headers)
	html = urlopen(req)
except HTTPError as e:
	err = e.read()
	code = e.getcode()
	print(err, code)

soup = BeautifulSoup(html, 'html.parser')

In [23]:
# 한, 미, 일, 중 채권
# rate_bonds_id = ['60', '1', '20', '54']

# 유럽 채권
rate_bonds_id = ['60', '1', '20', '54', '4', '40', '5', '3', '49', '9']
rate_bonds_link = []

In [24]:
for id_ in rate_bonds_id:
	rate_bond = soup.select(f'#rates_bonds_table_{id_} td.bold.left.noWrap.elp.plusIconTd > a')
	time.sleep(0.5)
	rate_bonds_link.extend(rate_bond)


In [25]:
rate_bonds_link

[<a href="/rates-bonds/south-korea-1-year-bond-yield" title="한국 1년">한국 1년</a>,
 <a href="/rates-bonds/south-korea-2-year-bond-yield" title="한국 2년">한국 2년</a>,
 <a href="/rates-bonds/south-korea-3-year-bond-yield" title="한국 3년">한국 3년</a>,
 <a href="/rates-bonds/south-korea-4-year-bond-yield" title="한국 4년">한국 4년</a>,
 <a href="/rates-bonds/south-korea-5-year-bond-yield" title="한국 5년">한국 5년</a>,
 <a href="/rates-bonds/south-korea-10-year-bond-yield" title="한국 10년">한국 10년</a>,
 <a href="/rates-bonds/south-korea-20-year-bond-yield" title="한국 20년">한국 20년</a>,
 <a href="/rates-bonds/south-korea-30-year" title="한국 30년">한국 30년</a>,
 <a href="/rates-bonds/south-korea-50-year" title="한국 50년">한국 50년</a>,
 <a href="/rates-bonds/u.s.-1-month-bond-yield" title="미국 1개월">미국 1개월</a>,
 <a href="/rates-bonds/u.s.-3-month-bond-yield" title="미국 3개월">미국 3개월</a>,
 <a href="/rates-bonds/u.s.-6-month-bond-yield" title="미국 6개월">미국 6개월</a>,
 <a href="/rates-bonds/u.s.-1-year-bond-yield" title="미국 1년">미국 1년</a>,
 <

In [26]:
rate_bonds = []
for each in rate_bonds_link:
	href = each['href']
	rate_bond = href.split('/')[-1]

	rate_bonds.append(rate_bond)

	middel_url = href.split('/')[-2]


In [27]:
investing_crawling_new(middel_url, rate_bonds, start_date)

전체 개수 145개


100%|██████████| 145/145 [49:43<00:00, 20.57s/it] 

성공 :['south-korea-1-year-bond-yield', 'south-korea-2-year-bond-yield', 'south-korea-3-year-bond-yield', 'south-korea-4-year-bond-yield', 'south-korea-5-year-bond-yield', 'south-korea-10-year-bond-yield', 'south-korea-20-year-bond-yield', 'south-korea-30-year', 'south-korea-50-year', 'u.s.-1-month-bond-yield', 'u.s.-3-month-bond-yield', 'u.s.-6-month-bond-yield', 'u.s.-1-year-bond-yield', 'u.s.-2-year-bond-yield', 'u.s.-3-year-bond-yield', 'u.s.-5-year-bond-yield', 'u.s.-7-year-bond-yield', 'u.s.-10-year-bond-yield', 'us-20-year-bond-yield', 'u.s.-30-year-bond-yield', 'japan-1-month', 'japan-3-month-bond-yield', 'japan-6-month-bond-yield', 'japan-9-month', 'japan-1-year-bond-yield', 'japan-2-year-bond-yield', 'japan-3-year-bond-yield', 'japan-4-year-bond-yield', 'japan-5-year-bond-yield', 'japan-6-year-bond-yield', 'japan-7-year-bond-yield', 'japan-8-year-bond-yield', 'japan-9-year-bond-yield', 'japan-10-year-bond-yield', 'japan-15-year-bond-yield', 'japan-20-year-bond-yield', 'japan-30

In [21]:
# 유럽 채권
url = 'https://kr.investing.com/rates-bonds/european-government-bonds?maturity_from=10&maturity_to=310'
try:
	headers = {'User-Agent': 'Chrome/107.0.5304.87'}
	req = urllib.request.Request(url, headers=headers)
	html = urlopen(req)
except HTTPError as e:
	err = e.read()
	code = e.getcode()
	print(err, code)

soup = BeautifulSoup(html, 'html.parser')

In [22]:
rate_bonds_link = soup.select(f'td.bold.left.noWrap.elp.plusIconTd > a')

In [23]:
rate_bonds = []
for each in rate_bonds_link:
	href = each['href']
	rate_bond = href.split('/')[-1]

	rate_bonds.append(rate_bond)

	middel_url = href.split('/')[-2]

In [24]:
rate_bonds

['greece-7-year-bond-yield',
 'greece-3-month-bond-yield',
 'greece-6-month-bond-yield',
 'greece-5-year-bond-yield',
 'greece-10-year-bond-yield',
 'greece-15-year-bond-yield',
 'greece-3-year-bond-yield',
 'netherlands-1-month',
 'netherlands-3-month-bond-yield',
 'netherlands-6-month-bond-yield',
 'netherlands-2-year-bond-yield',
 'netherlands-3-year-bond-yield',
 'netherlands-4-year-bond-yield',
 'netherlands-5-year-bond-yield',
 'netherlands-6-year-bond-yield',
 'netherlands-7-year-bond-yield',
 'netherlands-8-year-bond-yield',
 'netherlands-9-year-bond-yield',
 'netherlands-10-year-bond-yield',
 'netherlands-15-year',
 'netherlands-20-year-bond-yield',
 'netherlands-25-year',
 'netherlands-30-year-bond-yield',
 'norway-3-month-bond-yield',
 'norway-6-month-bond-yield',
 'norway-9-month-bond-yield',
 'norway-1-year-bond-yield',
 'norway-3-year-bond-yield',
 'norway-5-year-bond-yield',
 'norway-10-year-bond-yield',
 'germany-3-month-bond-yield',
 'germany-6-month-bond-yield',
 'ger

In [ ]:
investing_crawling_new(middel_url, rate_bonds, start_date)

전체 개수 327개


 54%|█████▎    | 175/327 [44:44<38:51, 15.34s/it]  


URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
url = 'https://kr.investing.com/central-banks/'

try:
	headers = {'User-Agent': 'Chrome/107.0.5304.87'}
	req = urllib.request.Request(url, headers=headers)
	html = urlopen(req)
except HTTPError as e:
	err = e.read()
	code = e.getcode()
	print(err, code)

soup = BeautifulSoup(html, 'html.parser')

In [ ]:
names = []
rates = []

interest_rate_name = soup.select('#curr_table > tbody > tr > td:nth-child(2) > a')
interest_rates = soup.select('#curr_table > tbody > tr > td:nth-child(3)')

for name in interest_rate_name:
	name_ = name.get_text()
	names.append(name_)

for rate in interest_rates:
	rate_ = rate.get_text()
	rates.append(rate_)

In [ ]:
names, rates

(['한국 은행',
  '연방준비은행',
  '유럽중앙은행',
  '영국은행',
  '스위스 국제은행',
  '호주 연방준비은행',
  '캐나다 은행',
  '일본 은행',
  '러시아 연방중앙은행',
  '인도 연방준비은행',
  '중국인민은행',
  '브라질 중앙은행'],
 ['3.00%',
  '4.00%',
  '2.00%',
  '3.00%',
  '0.50%',
  '2.85%',
  '3.75%',
  '-0.10%',
  '7.50%',
  '5.90%',
  '3.65%',
  '13.75%'])

In [ ]:
interest_rate_dict = {
	'name' : names
	, 'rate' : rates
}

interest_rate_df = pd.DataFrame(interest_rate_dict)
interest_rate_df

,name,rate
0,한국 은행,3.00%
1,연방준비은행,4.00%
2,유럽중앙은행,2.00%
3,영국은행,3.00%
4,스위스 국제은행,0.50%
5,호주 연방준비은행,2.85%
6,캐나다 은행,3.75%
7,일본 은행,-0.10%
8,러시아 연방중앙은행,7.50%
9,인도 연방준비은행,5.90%


In [ ]:
url = 'https://www.nanumtrading.com/%EC%9E%90%EB%A3%8C%EC%8B%A4/%EC%A3%BC%EC%9A%94%EA%B5%AD%EA%B0%80-%EA%B8%B0%EC%A4%80-%EA%B8%88%EB%A6%AC/'

In [ ]:
try:
	headers = {'User-Agent': 'Chrome/107.0.5304.87'}
	req = urllib.request.Request(url, headers=headers)
	html = urlopen(req)
except HTTPError as e:
	err = e.read()
	code = e.getcode()
	print(err, code)

soup = BeautifulSoup(html, 'html.parser')

In [ ]:
interest_rate_table = soup.select('table tr')
interest_rate_table

[<tr><td> </td><td style="text-align: center;"><span style="color: #8cc63f;"><strong>미국</strong></span></td><td style="text-align: center;"><span style="color: #8cc63f;"><strong>유로존</strong></span></td><td style="text-align: center;"><span style="color: #8cc63f;"><strong>일본</strong></span></td><td style="text-align: center;"><span style="color: #8cc63f;"><strong>영국</strong></span></td><td style="text-align: center;"><span style="color: #8cc63f;"><strong>스위스</strong></span></td><td style="text-align: center;"><span style="color: #8cc63f;"><strong>캐나다</strong></span></td><td style="text-align: center;"><span style="color: #8cc63f;"><strong>호주</strong></span></td><td style="text-align: center;"><span style="color: #8cc63f;"><strong>뉴질랜드</strong></span></td></tr>,
 <tr><td style="text-align: center;"><span style="color: #8cc63f;"><strong>12월</strong></span></td><td style="text-align: center;"><span style="color: #ff6600;"><strong>0.00~0.25%</strong></span></td><td style="text-align: center

In [ ]:
list_ = []
for row in interest_rate_table:
	row_ = row.get_text()
	list_.append(row_)

list_

['\xa0미국유로존일본영국스위스캐나다호주뉴질랜드',
 '12월0.00~0.25%2.50%0.10%2.00%0.50%1.50%4.25%5.00%',
 '11월-3.25%0.30%3.00%1.00%-5.25%-',
 '10월1.00%3.75%0.30%4.50%2.50%2.25%6.00%6.50%',
 '9월2.00%4.25%0.50%5.00%2.75%3.00%7.00%7.50%',
 '8월2.00%4.25%0.50%5.00%--7.25%-',
 '7월-4.25%0.50%5.00%-3.00%7.25%8.00%',
 '6월2.00%4.00%0.50%5.00%2.75%3.00%7.25%8.25%',
 '5월-4.00%0.50%5.00%--7.25%-',
 '4월2.00%4.00%0.50%5.00%-3.00%-8.25%',
 '3월2.25%4.00%0.50%5.25%2.75%3.50%7.25%8.25%',
 '2월-4.00%0.50%5.25%--7.00%-',
 '1월3.00%4.00%0.50%5.50%-4.00%-8.25%',
 '\xa0미국유로존일본영국스위스캐나다호주뉴질랜드',
 '12월0.00~0.25%1.00%0.10%0.50%0.25%0.25%-2.50%',
 '11월0.00~0.25%1.00%0.10%0.50%--3.50%-',
 '10월-1.00%0.10%0.50%-0.25%3.25%2.50%',
 '9월0.00~0.25%1.00%0.10%0.50%0.25%0.25%3.00%2.50%',
 '8월0.00~0.25%1.00%0.10%0.50%--3.00%-',
 '7월-1.00%0.10%0.50%-0.25%3.00%2.50%',
 '6월0.00~0.25%1.00%0.10%0.50%0.25%0.25%3.00%2.50%',
 '5월-1.00%0.10%0.50%--3.00%-',
 '4월0.00~0.25%1.25%0.10%0.50%-0.25%3.00%2.50%',
 '3월0.00~0.25%1.50%0.10%0.50%0.25%0.50%3.25%3.00%',
 '2월

In [ ]:
start_date = '2020-01-01'

calender_btn = '.DatePickerWrapper_icon-wrap__cwTu_'
calender_new_btn = '#flatDatePickerCanvasHol #datePickerIconWrap'

start_year_input = '.NativeDateInput_root__wbgyP > input'
start_year_new_input = '#startDate'

apply_btn = 'HistoryDatePicker_apply-button__fPr_G'
apply_new_btn = '#applyBtn'

csv_download_btn = '.download-data_download-data__jxNYT > a'
csv_download_new_btn = '#column-content > div.float_lang_base_2.downloadDataWrap > div > a'

bottom_modal = '.download-app-view_download-app-view__4d6Eq'
bottom_mpdal_btn = '.inv-button'

In [ ]:
all_button = [start_date,
              calender_btn,calender_new_btn,
              start_year_input,start_year_new_input,
              apply_btn,apply_new_btn,
              csv_download_btn,csv_download_new_btn,
              bottom_modal,bottom_mpdal_btn
              ]

In [ ]:
all_button

['2020-01-01',
 '.DatePickerWrapper_icon-wrap__cwTu_',
 '#flatDatePickerCanvasHol #datePickerIconWrap',
 '.NativeDateInput_root__wbgyP > input',
 '#startDate',
 'HistoryDatePicker_apply-button__fPr_G',
 '#applyBtn',
 '.download-data_download-data__jxNYT > a',
 '#column-content > div.float_lang_base_2.downloadDataWrap > div > a',
 '.download-app-view_download-app-view__4d6Eq',
 '.inv-button']

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
 
driver = webdriver.Chrome("C:\chromedriver.exe")
u = 'invasting site'
driver.get('https://kr.investing.com/')
 


C:\Users\TJ\AppData\Local\Temp\ipykernel_6576\4234165250.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\chromedriver.exe")
